In [14]:
import pandas as pd
import numpy as np

In [2]:
#cargamos los datos diarios de precios del Nasdaq Composite, desde 01/01/2015 a 26/09/2021
nas_hist = pd.read_csv('./Nasdaq_Comp.csv')

In [3]:
#Comprobamos que no faltan datos.

nas_hist[nas_hist.isna().any(axis=1)]

,Date,Open,High,Low,Close,Adj Close,Volume


In [4]:
nas_hist.head(10)

,Date,Open,High,Low,Close,Adj Close,Volume
0,2015-01-02,4760.240234,4777.009766,4698.109863,4726.810059,4726.810059,1435150000
1,2015-01-05,4700.339844,4702.770020,4641.459961,4652.569824,4652.569824,1794470000
2,2015-01-06,4666.850098,4667.330078,4567.589844,4592.740234,4592.740234,2167320000
3,2015-01-07,4626.839844,4652.720215,4613.899902,4650.470215,4650.470215,1957950000
4,2015-01-08,4689.540039,4741.379883,4688.020020,4736.189941,4736.189941,2105450000
5,2015-01-09,4744.470215,4744.709961,4681.240234,4704.069824,4704.069824,1715830000
6,2015-01-12,4714.069824,4715.810059,4650.649902,4664.709961,4664.709961,1861960000
7,2015-01-13,4708.740234,4751.339844,4624.279785,4661.500000,4661.500000,2162180000
8,2015-01-14,4610.759766,4655.370117,4595.979980,4639.319824,4639.319824,2073810000
9,2015-01-15,4657.459961,4663.959961,4567.390137,4570.819824,4570.819824,1976260000


In [5]:
#Nos vamos a poner a modificar datos, así que hacemos una copia
nas_hist_m=nas_hist.copy()

In [6]:
#Transformamos Date a formato fecha
nas_hist_m['Date']=pd.to_datetime(arg=nas_hist_m['Date'],format='%Y-%m-%d')

In [7]:
#Quitamos las columnas que no necesitamos (todas menos los cierres)
nas_hist_m = nas_hist_m.drop(labels=['Open','High','Low','Adj Close','Volume'], axis=1)

In [9]:
#añadimos columnas de variaciones de precio desde el día antes, semana antes y mes antes
nas_hist_m['C_dia_ant'] = nas_hist_m['Close'].shift(1)
nas_hist_m['C_dia_ant'].loc[[0]]=nas_hist_m['C_dia_ant'][1]

nas_hist_m['var_dia_ant']=100*nas_hist_m['Close']/nas_hist_m['C_dia_ant']-100


nas_hist_m['C_sem_ant'] = nas_hist_m['Close'].shift(5)

for i in range(0,5):
    nas_hist_m['C_sem_ant'].loc[[i]]=nas_hist_m['C_sem_ant'][i+5]

nas_hist_m['var_sem_ant']=100*nas_hist_m['Close']/nas_hist_m['C_sem_ant']-100


nas_hist_m['C_mes_ant'] = nas_hist_m['Close'].shift(20)

for i in range(0,20):
    nas_hist_m['C_mes_ant'].loc[[i]]=nas_hist_m['C_mes_ant'][i+20]

nas_hist_m['var_mes_ant']=100*nas_hist_m['Close']/nas_hist_m['C_mes_ant']-100

In [35]:
#ahora creamos el set de datos de verdad, que por ahora va a contener:
#los datos de variación de precios diarios de las últimas 4 semanas (20 días, el Nasdaq sólo cotiza 5 días/semana)
#los datos de variación de precios semanales desde 4 semanas antes a 12 semanas antes
#los datos de variación de precios mensuales desde 3 meses antes a 11 meses antes
#en este caso no se incluyen datos objetivo, ya que no queremos predecir nada del Nasdaq,
#únicamente son datos de referencia que creemos que pueden ayudar en las predicciones de las cripto
#todo esto desde el 29 de diciembre de 2015, más tarde separaremos los datos de training y de test
#el 29 de diciembre está elegido para estar seguros de tener datos anteriores al 1 de enero y poder usarlos
#como datos del 1 de enero para cruzar con los datos del bitcoin

columnasData=['varP0', 'varP1', 'varP2', 'varP3', 'varP4', 'varP5', 'varP6', 'varP7', 'varP8', 'varP9',
             'varP10', 'varP11', 'varP12', 'varP13', 'varP14', 'varP15', 'varP16', 'varP17', 'varP18', 'varP19',
             'varPs20', 'varPs25', 'varPs30', 'varPs35', 'varPs40', 'varPs45', 'varPs50', 'varPs55', 'varPs60',
             'varPm65', 'varPm85', 'varPm105', 'varPm125', 'varPm145', 'varPm165', 'varPm185', 'varPm205',
             'varPm225']
data_for_use = pd.DataFrame(columns=columnasData)


for i in range(0,nas_hist_m.shape[0]-1):
    if nas_hist_m['Date'][i]>pd.to_datetime(arg='2015-12-28',format='%Y-%m-%d'):
        #rellenamos las fechas en las que el Nasdaq no cotiza con los datos del día cotizado anterior
        if nas_hist_m['Date'][i]>pd.to_datetime(arg='2015-12-31',format='%Y-%m-%d') and \
            (nas_hist_m['Date'].loc[[i]].reset_index(drop=True)-pd.Timedelta(days=1) > \
            nas_hist_m['Date'].loc[[i-1]].reset_index(drop=True)).all():
            
            j=1
            while (nas_hist_m['Date'].loc[[i]].reset_index(drop=True)-pd.Timedelta(days=j) \
                > nas_hist_m['Date'].loc[[i-1]].reset_index(drop=True)).all():
                
                dtemp = dtemp.set_index(keys=[nas_hist_m['Date'].loc[[i-1]]+timedelta(days=j)])
                data_for_use = data_for_use.append(dtemp)
                j=j+1
        
        
        dtemp = pd.concat([pd.DataFrame(nas_hist_m['var_dia_ant'][i-19:i+1][::-1].values),
                            pd.DataFrame(nas_hist_m['var_sem_ant'][i-64:i-19][::-5].values),
                            pd.DataFrame(nas_hist_m['var_mes_ant'][i-244:i-64][::-20].values)],
                          ignore_index=True, axis=0).T

        dtemp = dtemp.set_index(keys=[nas_hist_m['Date'].loc[[i]]])
        
        new_cols = {x: y for x, y in zip(dtemp.columns, data_for_use.columns)}

        dtemp = dtemp.rename(columns=new_cols)

        data_for_use = data_for_use.append(dtemp)

In [36]:
data_for_use.head(20)

,varP0,varP1,varP2,varP3,varP4,varP5,varP6,varP7,varP8,varP9,...,varPs60,varPm65,varPm85,varPm105,varPm125,varPm145,varPm165,varPm185,varPm205,varPm225
2015-12-29,1.328106,-0.148559,0.050735,0.896207,0.647826,0.931121,-1.588584,-1.352363,1.516808,0.870918,...,0.454066,-2.622430,-6.162671,2.307139,-1.688684,3.237279,0.592642,-0.651702,4.174185,0.173345
2015-12-30,-0.824008,1.328106,-0.148559,0.050735,0.896207,0.647826,0.931121,-1.588584,-1.352363,1.516808,...,5.222075,-5.889204,-5.849134,2.377018,-0.986538,2.835166,-0.629788,1.873122,2.830724,0.466403
2015-12-31,-1.153606,-0.824008,1.328106,-0.148559,0.050735,0.896207,0.647826,0.931121,-1.588584,-1.352363,...,5.114538,-5.426346,-6.624535,2.472785,-1.509729,2.485470,-0.583369,2.569519,1.301263,2.635323
2016-01-01,-1.153606,-0.824008,1.328106,-0.148559,0.050735,0.896207,0.647826,0.931121,-1.588584,-1.352363,...,5.114538,-5.426346,-6.624535,2.472785,-1.509729,2.485470,-0.583369,2.569519,1.301263,2.635323
2016-01-02,-1.153606,-0.824008,1.328106,-0.148559,0.050735,0.896207,0.647826,0.931121,-1.588584,-1.352363,...,5.114538,-5.426346,-6.624535,2.472785,-1.509729,2.485470,-0.583369,2.569519,1.301263,2.635323
2016-01-03,-1.153606,-0.824008,1.328106,-0.148559,0.050735,0.896207,0.647826,0.931121,-1.588584,-1.352363,...,5.114538,-5.426346,-6.624535,2.472785,-1.509729,2.485470,-0.583369,2.569519,1.301263,2.635323
2016-01-04,-2.083319,-1.153606,-0.824008,1.328106,-0.148559,0.050735,0.896207,0.647826,0.931121,-1.588584,...,3.700948,-0.343822,-9.194890,2.162896,-0.481316,0.361345,0.151518,2.098587,1.918484,2.996464
2016-01-05,-0.237802,-2.083319,-1.153606,-0.824008,1.328106,-0.148559,0.050735,0.896207,0.647826,0.931121,...,3.970322,-2.587377,-7.586858,4.688217,-2.076447,0.406529,0.106647,2.391133,0.291294,4.705216
2016-01-06,-1.138121,-0.237802,-2.083319,-1.153606,-0.824008,1.328106,-0.148559,0.050735,0.896207,0.647826,...,2.606121,-0.543366,-6.386706,2.723063,-3.039186,2.019617,-0.022102,0.969270,0.728874,7.067004
2016-01-07,-3.025990,-1.138121,-0.237802,-2.083319,-1.153606,-0.824008,1.328106,-0.148559,0.050735,0.896207,...,1.200110,2.078171,-7.130312,0.917219,-1.668655,2.023808,-0.585312,1.490448,0.778895,5.715762


In [37]:
data_for_use.to_csv('nasdaq_data_for_use.csv')